# Turn Restrictions
This notebook is for donwloading OSM's turn restriction data which are stored as relations.

In [1]:
import requests
import json
from pathlib import Path 
import geopandas as gpd
import pandas as pd

In [4]:
fp = Path.home() / 'Documents/BikewaySimData/Projects/gdot/networks'

bikewaysim_studyarea = gpd.read_file(Path.home()/'Documents/BikewaySimData/Data/Study Areas/bikewaysim_studyarea.geojson')

links = gpd.read_file(fp/'reconciled_network.gpkg',layer='links_w_signals_elevation',mask=bikewaysim_studyarea)
links.to_crs('epsg:4326',inplace=True)


minx, miny, maxx, maxy = links.total_bounds

# Query

In [11]:
query = f"""
    [out:json]
    [timeout:30]
    ;
    (relation
        ["restriction"="no_u_turn"]
        ({miny},{minx},{maxy},{maxx});
    relation
        ["restriction"="no_left_turn"]
        ({miny},{minx},{maxy},{maxx});
    relation
        ["restriction"="only_straight_on"]
        ({miny},{minx},{maxy},{maxx});
    relation
        ["restriction"="no_right_turn"]
        ({miny},{minx},{maxy},{maxx});
    relation
        ["restriction"="only_left_turn"]
        ({miny},{minx},{maxy},{maxx});
    relation
        ["restriction"="no_straight_on"]
        ({miny},{minx},{maxy},{maxx});
    );
    out geom;
    """

url = "http://overpass-api.de/api/interpreter"
r = requests.get(url, params={'data': query})
result = r.json()

Desired format is: 
| Relation ID | From linkid | To linkid | restriction type |
| --- | --- | --- | --- |
| test | test | test | test |

In [85]:
turn_restrictions = []

for element in result['elements']:
    relation_id = element['id']
    restriction = element['tags']['restriction']

    #there can be multiple to restrictions
    to_way_ids = []

    for member in element['members']:
        if member['type'] == 'way':
            if member['role'] == 'from':
                from_way_id = member['ref']
            else:
                to_way_ids.append(member['ref'])

    turn_restrictions.append({
        'relation_id': relation_id,
        'restriction': restriction,
        'from_way_id': from_way_id,
        'to_way_id': to_way_ids
    })

turn_restrictions = pd.DataFrame().from_dict(turn_restrictions).explode('to_way_id')
turn_restrictions


,relation_id,restriction,from_way_id,to_way_id
0,975011,no_left_turn,62607074,311764284
1,977208,no_straight_on,9270296,63063102
2,1130607,only_straight_on,26800271,62607074
3,1130608,no_left_turn,1118916319,303454543
4,1687734,no_right_turn,9255787,123272761
...,...,...,...,...
295,16514302,no_right_turn,1216499871,1092954185
296,16767764,no_u_turn,1052987774,9246471
296,16767764,no_u_turn,1052987774,1227565871
297,17004254,no_left_turn,774938901,630048362


In [96]:
turn_restrictions['type'] = turn_restrictions['restriction'].apply(lambda text: text.split('_')[0])
turn_restrictions

,relation_id,restriction,from_way_id,to_way_id,type
0,975011,no_left_turn,62607074,311764284,no
1,977208,no_straight_on,9270296,63063102,no
2,1130607,only_straight_on,26800271,62607074,only
3,1130608,no_left_turn,1118916319,303454543,no
4,1687734,no_right_turn,9255787,123272761,no
...,...,...,...,...,...
295,16514302,no_right_turn,1216499871,1092954185,no
296,16767764,no_u_turn,1052987774,9246471,no
296,16767764,no_u_turn,1052987774,1227565871,no
297,17004254,no_left_turn,774938901,630048362,no


In [97]:
turn_restrictions.to_csv(fp.parent/'osm_turn_restrictions.csv',index=False)